# Derivations related to Cross Entropy Loss (Binary and Categorical)

### What is Binary Cross Entropy Loss?

Binary Cross Entropy Loss is defined as follows given our prediction vector y_{pred}$:

$$ BCE = -\sum_{i = 1}^{m}y^{(i)} \cdot \log(y_{pred}^{(i)}) + (1 - y^{(i)}) \cdot \log(1 - y_{pred}^{(i)}) $$

I will go over each term one-by-one to explain the intuition.
If the actual output $y^{(i)}$ is 1, then only the term $y^{(i)} \cdot \log(y_{pred}^{(i)})$ remains since $(1 - y^{(i)})$ results in 0. Then by taking the log of our probability output, a probability close to 0 will result in a value close to  $-\infty$ where as a probability close to 1 will result in a value close to 0. Since we were meant to predict 1, we get a very large negative number the further away we are from 1. 

In the other case, when the output $y^{(i)}$ is 0, then only the term $(1 - y^{(i)}) \cdot \log(1 - y_{pred}^{(i)})$ remains. If our probability $y_{pred}^{(i)}$ was close to 1, then subtracting it from 1 results in a number close to 0. Then when we take the log of that number we get a value clsoe to $-\infty$. However when our prediction was close to 0 (the true output), subtracting it from 1 results in a number close to 1, and taking the log of this results in a number close to 0. 

To summarize when we get the example very wrong, it results in a very large negative number, but when we get it close to being right it results in a small negative number. So if we sum it all up we get the total throughout all examples. Of course we typically think of minimizing loss functions (think of loss as positive) thus we negate the whole summation to make the entire term a positive number. Then, our goal will be to minimize this.

### Coding the BCE Formula 

In [2]:
import numpy as np
import sys
from pathlib import Path

# Add the directory containing your code file to sys.path
sys.path.append(str(Path().resolve().parent.parent))

from models.LogisticRegression import LogisticRegression

# CREATE SOME DUMMY DATA
x = np.linspace(0, 10, 20)
y = np.array([1 if el + 2*np.random.rand() else 0 for el in x])

# reshape the data to place into the model
X = np.reshape(x, (x.shape[0], 1))

# initialize our model (I will use my Logistic Regression here)
model = LogisticRegression(1)

First we need to find our predicted values $y_{pred}$ using our model (In this case Logistic Regression)

In [3]:
y_pred = model(X)
y_pred

array([0.54876573, 0.55214661, 0.55552267, 0.55889361, 0.56225914,
       0.56561894, 0.56897273, 0.5723202 , 0.57566106, 0.57899502,
       0.58232179, 0.58564109, 0.58895263, 0.59225613, 0.59555131,
       0.5988379 , 0.60211563, 0.60538422, 0.60864341, 0.61189295])

Now we just need to apply the formula above in code with our actual outputs $y$ and our predicted outputs $y_{pred}$.
<br>
It becomes quite simple with numpy, since we are able to take the element wise log of a numpy array as well as element wise multiply two numpy arrays and sum over them easily

In [4]:
def BCELoss(y, y_pred):
    return -np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

print(f"BCE Loss is: {BCELoss(y, y_pred)}")

BCE Loss is: 10.886927818646447


### Why Binary Cross Entropy? (Digging into the theory)

##### *(Feel free to skip this math if you are not interested)*

Let $y^{(i)} \in \{0, 1\}, \: y_{pred}^{(i)} \in [0, 1]$ be the actual output and predicted output for a given feature vector $x^{(i)} \in \mathbb{R}^{n}$ 
<br> Assume $P(y^{(i)} = 1 | x^{(i)}) = y_{pred}^{(i)}$. Then,
    $$ \implies P(y^{(i)} = 0 | x^{(i)}) = 1 - y_{pred}^{(i)} $$
    $$ \implies P(y^{(i)} | x^{(i)}) = (y_{pred}^{(i)})^{y^{(i)}} \cdot (1 - y_{pred}^{(i)})^{1 - y^{(i)}} $$
    $$ \implies y^{(i)} | x^{(i)} \sim Bernoulli(y_{pred}^{(i)}) $$

So now we know that our output given our input follows a bernoulli distribution with the probability parameter $p$ as our predicted output.

Let's define our predicted output as a function of our parameters for some fixed input. Rather than defining a hypothesis function of our input vector $x^{(i)}$, we define a function for a fixed x, and make our parameters the variable.

Let $$g^{(i)}: \mathbb{R}^{n} \rightarrow \mathbb{R} \\ \theta \rightarrow y_{pred}^{(i)}$$

Then we have the Likelihood as:

$$ \mathcal{L}(\theta) = \prod_{i = 1}^{m} P(y^{(i)} | x^{(i)}) $$

By taking the log of the likelihood, we then get a summation:

$$ \log{\mathcal{L}(\theta)} = l(\theta) = \sum_{i=1}^{m}\log((g^{(i)}(\theta))^{y^{(i)}} \cdot (1 - g^{(i)}(\theta))^{1 - y^{(i)}}) $$

$$ = \sum_{i=1}^{m}\log((g^{(i)}(\theta))^{y^{(i)}}) + \log((1 - g^{(i)}(\theta))^{1 - y^{(i)}}) $$
$$ = \sum_{i=1}^{m}y^{(i)} \cdot \log(g^{(i)}(\theta)) + (1 - y^{(i)}) \cdot \log(1 - g^{(i)}(\theta)) $$

Then we use Maximum Likelihood Estimation to estimate our parameters $\theta$

$$ = \underset{\theta}{\mathrm{argmax}} [\sum_{i=1}^{m}y^{(i)} \cdot \log(g^{(i)}(\theta)) + (1 - y^{(i)}) \cdot \log(1 - g^{(i)}(\theta))] $$
$$ = \underset{\theta}{\mathrm{argmin}} [-\sum_{i=1}^{m}y^{(i)} \cdot \log(g^{(i)}(\theta)) + (1 - y^{(i)}) \cdot \log(1 - g^{(i)}(\theta))] $$
$$ = \underset{\theta}{\mathrm{argmin}} [BCE] $$

Which is clearly equivalent to minimizing the binary cross entropy loss.

### The Derivative of BCE 

To perform an optimization algorithm (such as gradient descent) we require the calculation of the partial derivates of a loss function with respect to the parameters of the model. 

So far, we have defined $y^{(i)}, g^{(i)}(\theta), x^{(i)}$ to be the output, predicted output (as a function of our parameters) and our input vector respectively. In order to calculate the gradient of our loss function, we need to calculate the partial derivative w.r.t each of the parameters in the model. Lets first set our loss function to be a function of our parameters $J(\theta) = BCE$. Then for some arbitrary parameter $\theta_j$:

$$ \implies \frac{\partial J}{\partial \theta_{j}} = \frac{\partial}{\partial \theta_{j}}[-\sum_{i=1}^{m}y^{(i)} \cdot \log(g^{(i)}(\theta)) + (1 - y^{(i)}) \cdot \log(1 - g^{(i)}(\theta))] $$

$$ = -\sum_{i=1}^{m}\frac{\partial}{\partial \theta_{j}}[y^{(i)} \cdot \log(g^{(i)}(\theta)) + (1 - y^{(i)}) \cdot \log(1 - g^{(i)}(\theta))] $$
$$ = -\sum_{i=1}^{m}y^{(i)}\cdot \frac{1}{g^{(i)}(\theta)} \cdot \frac{\partial g^{(i)}}{\partial \theta_{j}} + (1 - y^{(i)})\cdot \frac{1}{(1 - g^{(i)}(\theta))} \cdot (-\frac{\partial g^{(i)}}{\partial \theta_{j}})$$
$$ = -\sum_{i=1}^{m}\frac{\partial g^{(i)}}{\partial \theta_{j}} \cdot (y^{(i)}\cdot \frac{1}{g^{(i)}(\theta)} - (1 - y^{(i)})\cdot \frac{1}{(1 - g^{(i)}(\theta))}) $$
$$ = -\sum_{i=1}^{m}\frac{\partial g^{(i)}}{\partial \theta_{j}} \cdot (\frac{y^{(i)}\cdot (1 - g^{(i)}(\theta)) - (1 - y^{(i)}) \cdot g^{(i)}(\theta)}{g^{(i)}(\theta) \cdot (1 - g^{(i)}(\theta))}) $$
$$ = -\sum_{i=1}^{m}\frac{\partial g^{(i)}}{\partial \theta_{j}} \cdot (\frac{y^{(i)} - g^{(i)}(\theta)}{g^{(i)}(\theta) \cdot (1 - g^{(i)}(\theta))}) $$

Now we can simplify this into a dot product of vectors since we are taking the sum of a product of elements. 
$$\text{Let  } y = (y^{(1)}, y^{(2)}, ..., y^{(m)}) \in \mathbb{R}^{m} \: \text{  and  } \: g(\theta) = (g^{(1)}(\theta), g^{(2)}(\theta), ..., g^{(m)}(\theta)) \in \mathbb{R}^{m} $$
$$\text{Let } \frac{\partial g}{\partial \theta_{j}} = (\frac{\partial g^{(1)}}{\partial \theta_{j}}, \frac{\partial g^{(2)}}{\partial \theta_{j}}, ..., \frac{\partial g^{(m)}}{\partial \theta_{j}}) \in \mathbb{R}^{m}$$

Then we can express this as the following:

$$ \frac{\partial J}{\partial \theta_{j}} = - \frac{\partial g}{\partial \theta_{j}}^T[(y - g(\theta)) \otimes (g(\theta) - g(\theta) \odot g(\theta))]

Where $\odot$ is the Hadamard product or element-wise multiplication and $\otimes$ is the Hadamard division or element-wise division

Let's refer to this set of Hadamard products and divisions as 
$y_{g} = (y - g(\theta)) \otimes (g(\theta) - g(\theta) \odot g(\theta)) \in \mathbb{R}^{m}$

Then we have: $$ \frac{\partial J}{\partial \theta_{j}} = - \frac{\partial g}{\partial \theta_j}^T y_{g} $$

Now to calculate the full gradient of the loss (which is a vector of all partial derivates) let us define the Jacobian matrix of $g$:
$$\text{Let } \frac{\partial g}{\partial \theta} = (\frac{\partial g}{\partial \theta_{1}} \rightarrow \frac{\partial g}{\partial \theta_{n}}) \in \mathbb{R}^{m \times n} $$

Finally we have:

$$ \nabla_{J}(\theta) = - \frac{\partial g}{\partial \theta}^T y_{g}

### Coding the derivative

In [5]:
import numpy as np
import sys
from pathlib import Path

# Add the directory containing your code file to sys.path
sys.path.append(str(Path().resolve().parent.parent))

from models.LogisticRegression import LogisticRegression

# CREATE SOME DUMMY DATA
x = np.linspace(0, 10, 20)
y = np.array([1 if el + 2*np.random.rand() else 0 for el in x])

# reshape the data to place into the model
X = np.reshape(x, (x.shape[0], 1))

# initialize our model (I will use my Logistic Regression here)
model = LogisticRegression(1)

After simplifying the calculation down to a matrix vector product, the coding becomes easy. We just have to code $-y_g$ and let our model calculate its own derivatives. Let's start by running a forward pass through our model

In [6]:
y_pred = model(X)
y_pred

array([0.65380716, 0.67051789, 0.68680838, 0.70265061, 0.7180205 ,
       0.73289793, 0.7472667 , 0.76111445, 0.77443249, 0.78721562,
       0.7994619 , 0.81117244, 0.8223511 , 0.83300421, 0.84314034,
       0.85276998, 0.86190528, 0.87055979, 0.87874822, 0.88648618])

Now we compute $-y_g = (y - g(\theta)) \otimes (g(\theta) - g(\theta) \odot g(\theta)) $

In [7]:
def grads(y_pred, y):
        return  (-1) * ((y - y_pred) / (y_pred - y_pred**2))

loss_grads = grads(y_pred, y)
loss_grads

array([-1.52950298, -1.49138453, -1.45601019, -1.42318242, -1.39271789,
       -1.36444648, -1.3382103 , -1.31386284, -1.29126813, -1.2703    ,
       -1.25084134, -1.2327835 , -1.21602562, -1.20047412, -1.18604217,
       -1.17264916, -1.1602203 , -1.14868618, -1.1379824 , -1.12804917])

Finally we need to multiply by the Jacobian, but since the Jacobian is model specific, we pass our loss gradients into our model to fully compute the grads.

In [26]:
final_grads = model.grads(X, loss_grads)
final_grads

array([-121.32886352,  -25.70463971])

As you can see we are left with a vector of gradients corresponding to each of the parameters of the model. We can then pass those gradients into an optimization algorithm such as stochastic gradient descent to apply them to the parameters.

#### Note: Categorical cross entropy can be derived in a similar pattern
However rather than outputting a probability of just one class, we are outputting a probability distribution (vector) over $k$ classes. This means that we assume the output $y^{(i)}$ given $x^{(i)}$ follows a multinoulli distribution, which is an extension of the bernoulli distribution to multiple dimensions. If you follow a similar derivation function you should end up minimizing what is called the Categorical Cross Entropy Loss. The gradient of this loss also follows a similar process, however you end up with a matrix of derivates where each column is a class and each row a parameter (since each class has it's own hypothesis function).